In [35]:
import torch
import torch.nn as nn
import pandas as pd
import torch.optim as optim
import os
import joblib
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from sklearn.metrics import classification_report

In [261]:
PATH = r'C:\Users\pka\kaggle\melanoma\input\siim-isic-melanoma-classification'
PATH_JPEG = r'C:\Users\pka\kaggle\melanoma\input\siim-isic-melanoma-classification\jpeg\train'
PATH_DCM = r'C:\Users\pka\kaggle\melanoma\input\siim-isic-melanoma-classification\train'
PATH_PKL_128 = r'C:\Users\pka\kaggle\melanoma\input\siim-isic-melanoma-classification\pkl_128'

class trainDataset(Dataset):

    def __init__(self, data, transform = None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
       
        name = self.data.image_name.values[idx]
        
        image = joblib.load(os.path.join(PATH_PKL_128, f'{name}.pkl'))

        if self.transform:
            image = self.transform(image)
        image = image.astype(np.float32)
        image /= 255
        image = np.transpose(image, (2,0,1))
        target = self.data.target.values[idx]
      
        
        return torch.tensor(image), torch.tensor(target, dtype = torch.float)

In [290]:
df = pd.read_csv(os.path.join(PATH, 'train_folds.csv')).head(100)

In [393]:
# SOURCE : https://github.com/digantamisra98/Mish

import torch
from torch import nn
import torch.nn.functional as F


@torch.jit.script
def mish(input):
    '''
    Applies the mish function element-wise:
    mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + exp(x)))
    See additional documentation for mish class.
    '''
    return input * torch.tanh(F.softplus(input))


class Mish(nn.Module):
    '''
    Applies the mish function element-wise:
    mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + exp(x)))
    Shape:
        - Input: (N, *) where * means, any number of additional
          dimensions
        - Output: (N, *), same shape as the input
    Examples:
        >>> m = Mish()
        >>> input = torch.randn(2)
        >>> output = m(input)
    '''
    def __init__(self):
        '''
        Init method.
        '''
        super().__init__()

    def forward(self, input):
        '''
        Forward pass of the function.
        '''
        return mish(input)


class Res50(nn.Module):
    
    def __init__(self, out):
        super(Res50, self).__init__()
        import torchvision.models as models
        m = models.resnext50_32x4d(pretrained=True)
        in_f = m.fc.in_features
        m.fc = nn.Identity()
        self.model = m
        self.h = nn.Sequential(
                                nn.Linear(in_f, 512),
                                Mish(),
                                nn.BatchNorm1d(512),
                                nn.Dropout(0.2),
                                nn.Linear(512, out)
                                )
    def forward(self, x):
        x = self.model(x)       
        x = self.h(x)
        return x
        

In [394]:
model = Res50(1)

In [395]:
tr_idx = np.where(df.fold != 1)
vl_idx = np.where(df.fold == 1)

In [396]:
from torch.utils.data import DataLoader
dl =  DataLoader(trainDataset(df.loc[tr_idx]), batch_size=10)
vl =  DataLoader(trainDataset(df.loc[vl_idx]), batch_size=10)

In [409]:
opt = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss()

In [411]:
for e in range(1):
    model.train()
    l = []
    v = []
    p = []
    t = []
    for image, target in tqdm(dl):
        model.zero_grad()
        
        y_ = model(image)      
        loss = criterion(
            #y_.view(-1),  #nn.BCEWithLogitsLoss()
            torch.sigmoid(y_).view(-1),    
            
            target
        )
        l.append(loss.item())
        loss.backward()
        opt.step()
        
    model.eval()
    with torch.no_grad():
        for image, target in tqdm(vl):
            y_ = model(image)            
            loss = criterion(
                #y_.view(-1), #nn.BCEWithLogitsLoss()
                torch.sigmoid(y_).view(-1),
                
                target)
            pred = torch.sigmoid(y_).view(-1).round()     
            p.append(pred)
            t.append(target)            
            v.append(loss.item())
        pp = torch.cat(p)
        tt = torch.cat(t)
        s = (pp == tt).sum().item()
        acc = s / len(tt)
    print(acc)
    print(classification_report(tt.numpy(), pp.numpy()))
    print(f'Loss train {np.mean(l)}, val loss {np.mean(v)}')    
        
        

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.29it/s]

0.0
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      18.0
         1.0       0.00      0.00      0.00       0.0

    accuracy                           0.00      18.0
   macro avg       0.00      0.00      0.00      18.0
weighted avg       0.00      0.00      0.00      18.0

Loss train 0.5182496441735162, val loss 0.7086738049983978
